# Silver Layer Data Processing

This notebook lists common commands for processing clean and transformed data in the Silver layer.

## 1. Initialize Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('silver-processing').getOrCreate()

## 2. Data Cleaning

In [ ]:
from pyspark.sql.functions import col, trim, to_date, regexp_replace, when

# Drop rows with null IDs
clean_df = raw_df.filter(col('id').isNotNull())

# Trim strings and normalize date and amount formats
clean_df = (clean_df.withColumn('name', trim(col('name')))
                     .withColumn('event_date', to_date('event_date', 'MM/dd/yyyy'))
                     .withColumn('amount', regexp_replace('amount', ',', '').cast('double')))

# Standardize country names
clean_df = clean_df.withColumn('country', when(col('country') == 'USA', 'United States').otherwise(col('country')))

## 3. Advanced Deduplication

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy('id').orderBy(col('update_time').desc())
unique_df = (clean_df.withColumn('rn', row_number().over(window_spec))
                       .filter(col('rn') == 1)
                       .drop('rn'))

## 4. Data Enrichment

In [ ]:
# Join with reference data
enriched_df = unique_df.join(dim_country, on='country_code', how='left')

# Add derived columns
enriched_df = enriched_df.withColumn('age', 
                                     datediff(current_date(), col('birth_date')) / 365)

## 5. Structural Transformations

In [ ]:
from pyspark.sql.functions import from_json, explode, year
from pyspark.sql.types import StructType, StructField, StringType

# Flatten nested JSON column
json_schema = StructType([StructField('field1', StringType(), True),
                          StructField('field2', StringType(), True)])
flat_df = enriched_df.withColumn('json', from_json('json_col', json_schema))
flat_df = flat_df.select('*', 'json.*').drop('json')

# Pivot example
pivot_df = flat_df.groupBy('id').pivot('status').count()

# Apply strict schema
cast_df = flat_df.withColumn('year', year('event_date').cast('int'))

## 6. Filtering

In [ ]:
filtered_df = cast_df.filter(col('is_active') == True)

## 7. Data Quality Validation

In [ ]:
from pyspark.sql.functions import count, expr

# Validate that amount is positive
invalid_amount = filtered_df.filter(col('amount') <= 0).count()

# Record metrics
row_count = filtered_df.count()

validation_metrics = spark.createDataFrame([(row_count, invalid_amount)], ['row_count', 'invalid_amount'])

## 8. Optimization for Queries

In [ ]:
optimized_df = filtered_df.repartition('country').cache()

# Write compressed Delta tables
optimized_df.write.format('delta').mode('overwrite').option('compression', 'zstd').save('/mnt/silver/clean_table')